In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pylab
import re
import string
from operator import itemgetter
from pandas import *

# ---------------------------------------------------------------------------
# Data Preparation
# ---------------------------------------------------------------------------

Global variables

In [2]:
state_rank_count = 50

Read in opendata_projects file and donation_counts project file, google queries data, and outside_dat

In [3]:
projects = pd.read_csv('../Data/opendata_projects.csv', index_col = False, dtype = {'school_zip': np.str_})

In [4]:
donations = pd.read_csv('../Data/donations_counts.csv')

In [5]:
trends = pd.read_csv('../Data/google_queries.csv')

In [6]:
outside_dat =  pd.read_csv('../IndividualScripts/outside_dat_merge/outside_datmay16.csv', 
                           dtype = {'school_zip': np.str_, 'med_inc': np.float64, 'pop': np.float64, 
                                    'party': np.str_})

Use "shipping cost" variable to created binary variable indicating whether a project has free shipping

In [7]:
shipping = pd.DataFrame(projects['vendor_shipping_charges'])
# if shipping_charges = 0 , t
# if shipping_charges > 0 , f
shipping[shipping == 0] = -1
shipping[shipping > 0] = 'No'
shipping[shipping == -1] = 'Yes'

shipping.columns.values[0]='free_shipping'

Function to count the number of days between start_date and complete_date

In [8]:
from dateutil.parser import parse
import math
import time

def day_count(date0, date1):
    if isinstance(date0, float) or isinstance(date1, float):
        return 'NaN'
    try:
        delta = parse(date0) - parse(date1)
        delta = delta.days
        if delta < 0:
            delta = 'NaN'       
        elif delta == 0:
            delta = 1      
        else:
            delta = delta
    except Exception, e:
        print date1, type(date1), date0, type(date0)
        raise e
    return delta

Apply dayCount function over dataframe

In [9]:
def date_calc(row):
    return day_count(row['date_completed'], row['date_posted'])
date_diff = projects.apply(date_calc, 1)

Create single vector dataframe of days to completion

In [10]:
days_to_comp = pd.DataFrame({'days_to_completion': pd.Series(date_diff)})

Function to count the number of years after 2000 in which a project was posted

In [11]:
def years_since_2000(date0):
    if isinstance(date0, float):
        return 'NaN'
    try:
        delta = parse(date0).year - 2000
        if delta < 0:
            delta = 'NaN'       
        elif delta == 0:
            delta = 1      
        else:
            delta = delta
    except Exception, e:
        print date0, type(date0)
        raise e
    return delta

Apply years_since_2000 function over dataframe

In [12]:
def year_calc(row):
    return years_since_2000(row['date_posted'])
year_diff = projects.apply(year_calc, 1)

In [13]:
years_since_2000 = pd.DataFrame({'years_since_2000': pd.Series(year_diff)})

Subset opendata_projects to include only live projects

In [14]:
live_projects = projects[projects.funding_status == 'live']

Add column of current date so we can count open days

In [15]:
live_projects['current_date'] = '2014-11-10'

/home/brianna/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Apply dayCount over live projects

In [16]:
def date_calc2(row):
    return day_count(row['current_date'], row['date_posted'])
date_diff2 = live_projects.apply(date_calc2, 1)

Create single vector dataframe of days open. If project live fewer than 30 days, set days live to NA

In [17]:
days_live = pd.DataFrame({'days_open': pd.Series(date_diff2)})

Add the dataframes containing days_to_completion and days_open to projects dataframe

In [18]:
projects = pd.concat([projects, days_to_comp, days_live, shipping, years_since_2000], axis = 1)

Consider all reallocated projects and all projects live for more than 30 days as not funded at all. Delete all remaining live projects (which have been live for less than 30 days)

In [19]:
projects.ix[projects.days_to_completion <= 30, 'funded_by_30'] = 'Yes'
projects.ix[projects.days_to_completion > 30, 'funded_by_30'] = 'No'
projects.ix[projects.funding_status=='reallocated', 'funded_by_30'] = 'No'
projects.ix[projects.days_open > 30, 'funded_by_30'] = 'No'

projects = projects[projects.funded_by_30.notnull()]

Create donor interest parameters by subject and poverty level

By SUBJECT

In [20]:
# Deal with missing values
index = np.where(projects['primary_focus_subject'].isnull())[0]
projects.loc[index, 'primary_focus_subject'] = 'Missing'
total_size = len(projects) 
total_donors = projects['num_donors'].sum()

subjects = projects['primary_focus_subject']
subjects = subjects.unique()
subjects.sort()

In [21]:
num_donors_sub = projects.groupby(['primary_focus_subject']).sum()['num_donors']
subjects_size = projects.groupby('primary_focus_subject').size()
subjects_prop = subjects_size / total_size
scaled_interest_par_sub = (num_donors_sub/total_donors)**2 /subjects_prop 

In [22]:
df = pd.DataFrame({'primary_focus_subject': subjects, 'scaled_interest_par_sub' : scaled_interest_par_sub.values})
projects = pd.merge(projects, df, left_on = 'primary_focus_subject', right_on='primary_focus_subject', how='left')

By POVERTY LEVEL

In [23]:
# Deal with missing values
index = np.where(projects['poverty_level'].isnull())[0]
projects.loc[index, 'poverty_level'] = 'Missing'
total_size = len(projects) 
total_donors = projects['num_donors'].sum()

poverty = projects['poverty_level']
poverty = poverty.unique()
poverty.sort()

In [24]:
num_donors_pov = projects.groupby(['poverty_level']).sum()['num_donors']
poverty_size = projects.groupby('poverty_level').size()
poverty_prop = poverty_size / total_size
scaled_interest_par_pov = (num_donors_pov/total_donors)**2 /poverty_prop

In [25]:
df = pd.DataFrame({'poverty_level': poverty, 'scaled_interest_par_pov' : scaled_interest_par_pov.values})
projects = pd.merge(projects, df, left_on = 'poverty_level', right_on='poverty_level', how='left')

In [26]:
#Replace missing values as NaN
projects = projects.replace('Missing', np.nan)

Combine city, state into city_state variable in projects dataframe

In [27]:
projects['city_state'] = projects['school_city'] + ', ' + projects['school_state']

Get the count of the completed projects from each city_state
and create levels from the top 50, and group all remaining into 51st category

In [28]:
comp = projects[projects.funding_status == 'completed']
counts = comp['city_state'].value_counts()
ranks = pd.DataFrame([x+1 for x in range(counts.shape[0])], columns = ['completion_rank'])
names = pd.DataFrame(counts.index, columns = ['city_state'])

In [29]:
ranks = pd.concat([names, ranks], axis = 1)
ranks['city_state_cat'] = ranks['city_state']

In [30]:
ranks['city_state_cat'][ranks.completion_rank > state_rank_count] = 'Other'

/home/brianna/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Separate city_state into categories based on the counts of funded projects in each city_state. Thresholds include: 10 or fewer, 11 to 50, 51 to 100, 101 to 200, 201 to 500, and greater than 500. 

In [31]:
max_count = max(counts)
counts_cut = counts
counts_cut[(counts_cut>=1) & (counts_cut<=10)] = 1
counts_cut[(counts_cut>10) & (counts_cut<=50)] = 2
counts_cut[(counts_cut>50) & (counts_cut<=100)] = 3
counts_cut[(counts_cut>100) & (counts_cut<=200)] = 4
counts_cut[(counts_cut>200) & (counts_cut<=500)] = 5
counts_cut[(counts_cut>500) & (counts_cut<=max_count)] = 6

cut = pd.DataFrame(counts_cut.values, columns = ['city_state_counts'])

In [32]:
ranks = pd.concat([ranks, cut], axis = 1)

Join the ranks table and projects table on the city_state columns

In [33]:
projects = projects.merge(ranks, how = 'right', on = 'city_state')

Join the donations table and projects table on the city_state column

In [34]:
projects = projects.merge(donations, how = 'right', on = 'city_state')

In [35]:
# Function to process Google queries data
def date_conv(row):
    date = row.isocalendar()
    year_week = str(date[0]) + " " + str(date[1])
    return year_week

In [36]:
date_posted = pd.to_datetime(projects['date_posted'])
projects['year_week'] = date_posted.apply(date_conv, 1)

projects = pd.merge(projects, trends, on = 'year_week', how='left')

Merge outside_data with projects

In [37]:
projects = projects.merge(outside_dat, on = 'school_zip', how='left')

Drop last 20986 rows because those rows only have outside data

In [38]:
projects = projects.drop(projects.tail(20986).index)

In [39]:
#Function to remove projects posted prior to 2010
def sub_projects(date):
    if isinstance(date, float):
        return 'NaN'
    try:
        date = parse(date)
        year = date.year
    except Exception, e:
        print date, type(date)
        raise e
    if year >= 2010:
        return 'Keep'
    else: 
        return 'Drop'
    
projects['keep_drop'] = [sub_projects(x) for x in projects.date_posted]
projects = projects[projects.keep_drop == 'Keep']

# ---------------------------------------------------------------------------
# Generate test set / training set
# ---------------------------------------------------------------------------

Create test set (~20%) by selecting projects posted from Nov 2013 onwards. Then create training (~80%)

In [40]:
def split_train_test(date):
    if isinstance(date, float):
        return 'NaN'
    try:
        date = parse(date)
        day = date.day
        month = date.month
        year = date.year
    except Exception, e:
        print date, type(date)
        raise e
    #if year == 2014 or (year == 2013 and (month == 11 or month == 12)):
    if year == 2014 and month >= 4:
        return 'Test'
    else:
        return 'Train'

In [41]:
projects['train_test_label'] = [split_train_test(x) for x in projects.date_posted]

In [42]:
train = projects[projects.train_test_label == 'Train']
test = projects[projects.train_test_label == 'Test']

Create three new trainings sets, >=2010, >=2008, >=2006

Remove unnecessary columns from projects dataframe

Removed: _projectid, _teacher_acctid, _schoolid, school_ncesid, school_city, school_state, school_zip, school_district, school_county, school_kipp, school_charter_ready_promise, primary_focus_area, secondary_focus_area, sales_tax, payment_processing_charges, fulfillment_labor_materials, total_price_including_optional_support, students_reached, total_donations, num_donors, eligible_double_your_impact_match, eligible_almost_home_match, funding_status, date_posted, date_completed, date_thank_you_packet_mailed, date_expiration, secondary_focus_subject, city_state, vendor_shipping_charges, city_state_cat, keep_drop

In [43]:
drop_cols = ['_projectid', '_teacher_acctid', '_schoolid', 'school_ncesid', 'school_city', 'school_state', 'school_zip', 
             'school_district', 'school_county', 'school_kipp', 'school_charter_ready_promise', 'primary_focus_area', 
             'secondary_focus_area', 'completion_rank', 'sales_tax', 'payment_processing_charges', 'fulfillment_labor_materials', 
             'total_price_including_optional_support', 'students_reached', 'total_donations', 'num_donors', 'eligible_double_your_impact_match', 
             'eligible_almost_home_match', 'funding_status', 'date_posted', 'date_completed', 'date_thank_you_packet_mailed',
             'date_expiration', 'secondary_focus_subject', 'Unnamed: 0', 'city_state', 'donor_counts', 'days_open', 
             'days_to_completion', 'vendor_shipping_charges', 'city_state_cat', 'keep_drop', 'year_week', 'city_state_counts']

In [44]:
projects = projects.drop(drop_cols, axis = 1)
train = train.drop(drop_cols, axis = 1)
test = test.drop(drop_cols, axis = 1)

Write full dataset to csv

In [45]:
projects.to_csv('../Data/projects_prelim_data.csv', index = False)

Drop train_test_label column from training and testing sets

In [46]:
drop_cols2 = ['train_test_label'] 
train = train.drop(drop_cols2, axis = 1)
test = test.drop(drop_cols2, axis = 1)

Write separate training/testing sets to csvs

In [47]:
train.to_csv('../Data/training_set.csv', index = False)
test.to_csv('../Data/testing_set.csv', index = False)

In [50]:
train.head

<bound method DataFrame.head of         school_latitude  school_longitude school_metro school_charter  \
8267          40.822187        -73.855974        urban              f   
8268          40.822187        -73.855974        urban              f   
8299          40.822187        -73.855974        urban              f   
8497          40.822277        -73.894063        urban              f   
8505          40.851853        -73.910253        urban              f   
8516          40.844243        -73.907793        urban              f   
8743          40.844243        -73.907793        urban              f   
8824          40.822187        -73.855974        urban              f   
8831          40.879883        -73.888962        urban              f   
8835          40.844243        -73.907793        urban              f   
8912          40.874756        -73.867821        urban              f   
9020          40.815018        -73.856094        urban              f   
9064          40.86